### Preset editor: Operator (AM)

This notebook saves mapped parameter values back into Ableton Live’s native preset format. Because most closed‑source plugins do not document their preset serialization, we first decompress and inspect the preset structure and then write our values into the appropriate XML nodes.

In the context of the paper, `preset_editor_xx` notebooks perform this export step for different synthesizers/plugins. Here, `preset_editor_operator_additive` targets Ableton Operator for an additive synthesis configuration (4 parallel oscillators), writing values computed in `parameter_operator_additive` pipeline. We start from a blank/default preset (`dummy.adv`) and modify it. A comparison table verifies the written parameters.

### Workflow

- **Decompress preset**: Load the compressed `.adv` preset and decompress it to XML (`.xml`).
- **Load mapped values**: Read tabular parameters from `tsv/final_values_am_operator.tsv` and normalize waveform names to Operator waveform IDs.
- **Update XML nodes**: For each `Operator.i`, set `Coarse/Manual@Value`, `Fine/Manual@Value`, `Volume/Manual@Value`, and `WaveForm/Manual@Value` from the table/mapping.
- **Save updated preset XML**: Write to `xml_presets/updated_amtest.xml` (or similar) with XML declaration.
- **Verify changes**: Extract before/after values and display a comparison DataFrame.
- **Re-compress to preset**: Gzip the updated XML back to `.adv` (e.g., `xml_presets/compressed_amttest.adv`) for use in Ableton Live.

Note: This mirrors the FM workflow with specific additions (waveform mapping).


In [1]:
import gzip

# Function to decompress a gzip file
def decompress_gzip(file_path, output_path):
    with gzip.open(file_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            f_out.write(f_in.read())

# Paths to the input and output files
input_path = 'xml_presets/dummy.adv'
output_path = 'xml_presets/dummy.xml'

# Decompress the gzip file
decompress_gzip(input_path, output_path)

print(f"Decompressed file saved to {output_path}")


Decompressed file saved to xml_presets/dummy.xml


In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import io
import re

# --- 1. Define TSV Data and Waveform Mapping ---

tsv_filename = 'tsv/final_values_am_operator.tsv'
df = pd.read_csv(tsv_filename, sep='\t')


# Mapping your waveform names to the best Ableton Operator equivalents
WAVEFORM_MAP = {
    'sinewave': '0',       # sine_wave -> "Sine"
    'squarewave': '18',      # square_wave -> "Sqr D"
    'trianglewave': '19',    # triangle_wave -> "Tri"
    'sawtoothwave': '10',    # sawtooth_wave -> "Saw D"
    'noisewave': '21'        # noise_wave -> "Noise White"
}

def get_waveform_value(name):
    """Normalizes the waveform name and finds its corresponding integer value."""
    normalized_name = re.sub(r'[\s_-]', '', name.lower())
    return WAVEFORM_MAP.get(normalized_name, '0') # Default to Sine if not found


# --- 2. Extract All Necessary Values from the DataFrame ---

# Extract new values for ALL parameters from the DataFrame
new_values = {}
for i in range(len(df)):
    new_values[f"Operator.{i}"] = {
        "coarse": int(df.loc[i, "Coarse Tuning"]),
        "fine": int(df.loc[i, "Fine Tuning"]),
        # **Using "Amplitude" column as it's the standard linear value (0-1) for volume**
        "volume": df.loc[i, "Amplitude"],
        "waveform": get_waveform_value(df.loc[i, "Waveform"])
    }

# --- 3. Load and Modify the XML File ---

# Load the XML file to be modified
# Make sure the path is correct for your setup
file_path = 'xml_presets/dummy.xml'
tree = ET.parse(file_path)
root = tree.getroot()

# Update the Algorithm to 4 parallel oscillators (Value 10)
algorithm = root.find('.//Globals/Algorithm/Manual')
if algorithm is not None:
    algorithm.set('Value', '10')
    print(f"Algorithm set to 10 (4 parallel oscillators)")
else:
    print("Warning: Algorithm tag not found in XML.")

# Update the values for each operator based on the extracted data
for operator_tag, values in new_values.items():
    operator = root.find(f'.//{operator_tag}')
    if operator is not None:
        # Update Coarse
        coarse_el = operator.find('.//Coarse/Manual')
        if coarse_el is not None:
            coarse_el.set('Value', str(values["coarse"]))

        # Update Fine
        fine_el = operator.find('.//Fine/Manual')
        if fine_el is not None:
            fine_el.set('Value', str(values["fine"]))

        # Update Volume
        volume_el = operator.find('.//Volume/Manual')
        if volume_el is not None:
            volume_el.set('Value', str(values["volume"]))

        # Update Waveform
        waveform_el = operator.find('.//WaveForm/Manual')
        if waveform_el is not None:
            waveform_el.set('Value', values["waveform"])
    else:
        print(f"Warning: {operator_tag} not found in XML.")

# --- 4. Save the Updated XML File ---

# Save the updated XML to a new file
updated_file_path = 'xml_presets/updated_additive_preset_full.xml'
tree.write(updated_file_path, xml_declaration=True, encoding='utf-8', method="xml")

print(f"Successfully updated preset saved at: {updated_file_path}")


# --- 5. Verification Step (Optional but Recommended) ---
def extract_all_values(xml_root):
    data = {}
    for i in range(4):
        op_tag = f'Operator.{i}'
        operator = xml_root.find(f'.//{op_tag}')
        if operator is not None:
            data[op_tag] = {
                'coarse': operator.find('.//Coarse/Manual').get('Value'),
                'fine': operator.find('.//Fine/Manual').get('Value'),
                'volume': operator.find('.//Volume/Manual').get('Value'),
                'waveform': operator.find('.//WaveForm/Manual').get('Value')
            }
    return data

original_values = extract_all_values(ET.parse(file_path).getroot())
updated_values = extract_all_values(ET.parse(updated_file_path).getroot())

# Create a DataFrame to compare original and updated values
comparison_df = pd.DataFrame({
    "Original Coarse": {op: original_values[op]['coarse'] for op in original_values},
    "Updated Coarse": {op: updated_values[op]['coarse'] for op in updated_values},
    "Original Fine": {op: original_values[op]['fine'] for op in original_values},
    "Updated Fine": {op: updated_values[op]['fine'] for op in updated_values},
    "Original Volume": {op: original_values[op]['volume'] for op in original_values},
    "Updated Volume": {op: updated_values[op]['volume'] for op in updated_values},
    "Original Waveform": {op: original_values[op]['waveform'] for op in original_values},
    "Updated Waveform": {op: updated_values[op]['waveform'] for op in updated_values}
})

# Display the comparison DataFrame
comparison_df

Algorithm set to 10 (4 parallel oscillators)
Successfully updated preset saved at: xml_presets/updated_additive_preset_full.xml


,Original Coarse,Updated Coarse,Original Fine,Updated Fine,Original Volume,Updated Volume,Original Waveform,Updated Waveform
Operator.0,1,1,0,0,0.1258925349,0.9480487628150518,0,10
Operator.1,1,1,24,108,0.1000000015,0.1519455665280356,0,0
Operator.2,1,1,514,656,0.1083927006,0.3321346643253397,0,0
Operator.3,0,2,13,460,0.5623413324,0.1024835139694349,0,0


In [3]:
import gzip

# Function to compress an XML file to gzip format
def compress_to_gzip(input_path, output_path):
    with open(input_path, 'rb') as f_in:
        with gzip.open(output_path, 'wb') as f_out:
            f_out.writelines(f_in)

# Paths to the input (edited XML) and output (compressed) files
input_path = 'xml_presets/updated_additive_preset_full.xml'  # Your edited XML file
output_path = 'xml_presets/compressed_amttest4.adv'   # The gzip output file

# Compress the XML file to gzip format
compress_to_gzip(input_path, output_path)

print(f"Compressed file saved to {output_path}")


Compressed file saved to xml_presets/compressed_amttest4.adv
